# 4.1 补充metadata信息

In [16]:
# 1. 添加文档到chromadb当中
# 2. 根据问题进行检索并输出答案

import chromadb
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser

def get_metadata_str(metadata):
    if metadata is None:
        return ""
    _str = ""
    # (key, value)
    for key, value in metadata.items():
        _str += f"{key}:{value}\n"
    return _str

class RagLangChain():
    def __init__(self, 
                 collection_name: str = "rag-video-collection", 
                 host: str = "localhost",
                port: int = 8000):
        self.collection_name = collection_name
        self.host = host
        self.port = port

    def __get_vector_store(self):
        chromadb_client = chromadb.HttpClient(host=self.host, port=self.port)
        embedding_fn = OllamaEmbeddings(model="nomic-embed-text:latest")
        
        vector_store = Chroma(collection_name=self.collection_name,
                              client=chromadb_client, 
                              embedding_function=embedding_fn
                             )
        return vector_store

    def delete_collection(self):
        vector_store = self.__get_vector_store()
        vector_store.delete_collection()
        
    def add_file(self, file_path: str, metadata = None):
        loader = TextLoader(file_path)
        
        docs = loader.load()
        text_spliter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
        all_splits = text_spliter.split_documents(docs)

        if metadata is not None:
            for split_item in all_splits:
                split_metadata = split_item.metadata
                split_item.metadata = {**split_metadata, **metadata}
        
        vector_store = self.__get_vector_store()
        ids = vector_store.add_documents(documents=all_splits)
        return ids

    def __query_vector(self, info):
        vector_store = self.__get_vector_store()
        retriever = vector_store.as_retriever()
        docs = retriever.invoke(info["query"])
        for doc_item in docs:
            print(doc_item)
            print("*" * 80)
        # docs_str = "\n\n".join(doc.page_content for doc in docs)
        docs_str = "\n\n".join(f"""
{get_metadata_str(doc.metadata)}
{doc.page_content}""" for doc in docs)
        print(f"@@@@@{docs_str}")
        return docs_str
        
    def query(self, question: str):
        prompt = ChatPromptTemplate.from_template("""
你是一个问答机器人。你的任务是根据下述给定的已知信息回答用户问题。

已知信息:
{context}
用户问题：
{query}
如果已知信息不包含用户问题的答案，或者已知信息不足以回答用户的问题，请直接回复"我无法回答您的问题"。
请不要输出已知信息中不包含的信息或答案。
请用中文回答用户问题。
""")
        
        llm = ChatOllama(model="qwen2.5:latest")
        
        output_parser = StrOutputParser()
        # 返回值: {"query": "XXXX", "context": "XXXXX2"}
        chain = ( {"context": self.__query_vector, "query": lambda x: x["query"]} | prompt | llm | output_parser)
        
        result = chain.invoke({"query": question})
        return result


In [17]:
rag_chain = RagLangChain()

In [18]:
rag_chain.delete_collection()

In [19]:
import chromadb
client = chromadb.HttpClient(host="localhost", port=8000)
client.list_collections()
# collction_01 = client.get_collection(name="rag-video-collection")
# collction_01.get(ids=["0ff540ea-3e4b-4732-ba5c-1d3864f30914"])

[Collection(name=my_collection_02),
 Collection(name=similarity_collection_cosine),
 Collection(name=similarity_collection_ip),
 Collection(name=similarity_collection_01),
 Collection(name=my_collection),
 Collection(name=my_collection_03),
 Collection(name=query_collection_01),
 Collection(name=query_video_collection)]

In [20]:
metadata = {
    "文章标题": "科技行业2025年展望",
    "作者": "沈岱,马智焱,黄佳琦",
    "发表时间": "2024 年 12 月 13 日",
}
rag_chain.add_file(file_path="科技行业 2025 年展望.txt", metadata=metadata)

['968d0465-4d3f-4b5c-8326-58cecc4aa70c',
 '87b2c55c-0315-4243-872b-5ec89eaddbcb',
 'a93ae3d9-446a-4cdc-8956-0ac2ba388b73',
 '50a6ec09-14dc-452c-888a-ad926b607210',
 'a62e2881-0384-4dc5-b835-23809c19f344',
 '83600e1a-83c1-446d-b8f9-9767be2e1977',
 'bd1bdb08-0dab-4575-8019-c3014c04a618',
 'b61e7961-f2e7-4457-b8ea-2521b3c63df5',
 'd39565da-7f6f-4c07-9ce1-c2e1fe4d40f6',
 'b336442e-c280-4c21-ace5-b35e16f1b8cc',
 '5b7abee2-4294-4cf7-9e2e-ec2cb5e23e2c',
 '3e19abdb-37c7-4cde-bcdb-53de620afad5',
 '65980f7e-6db1-4c90-9e3a-a2d4604508df',
 'a0f37d09-f5c3-497d-a7a6-8b5347d3416f',
 '3ef9ff74-f9c8-45c7-bed5-859eba8ced96',
 'f67e88cc-8c38-4e28-b0bd-63e071db6400',
 '26a2bfa6-8590-4071-973b-ed0a9fb1ba8b',
 '66ea2173-7566-4c4a-9a12-fdb9847d4198',
 '8b140ef9-4737-4e04-826c-bb4d3e46526d',
 '4ab29195-e276-4ec1-a90c-157319238a62',
 'e18730fa-9b43-4af5-ae02-bbd6b987becd',
 '58b1db67-84ee-4b40-8de5-c301902ee07c',
 '78a6ff51-e183-40a7-a38b-3fb989a0a9c9']

In [21]:
rag_chain.query(question="2024年三季度全球智能手机出货量 多少部?")

page_content='三、智能手机:2024 年、2025 年连续两年保持增长
预计 2025 年全球智能手机出货量将同比增长 2%
根据 IDC 数据，今年三季度全球智能手机出货量 3.15 亿部，环比增长 8%， 同比增长4%(图表 34)。虽然同比增速较一二季度下行，但是同比增速略 好于我们此前预测。这与我们与智能手机供应链沟通下来的情况类似，比半 年报看到的行业需求情况略好。' metadata={'source': '科技行业 2025 年展望.txt', '作者': '沈岱,马智焱,黄佳琦', '发表时间': '2024 年 12 月 13 日', '文章标题': '科技行业2025年展望'}
********************************************************************************
page_content='总体来看，我们预期今年全球智能手机出货量将达到 12.2 亿部，同比增长5.1%。我们略微上调全年出货量预测。全球智能手机出货量在今年取得较好 增长，在2023 年接近冰点的出货量基础上实现了较好的复苏。
我们认为 2025 年有望延续 2024 年的复苏势头，预计 2025 年全球智能手机 出货量将达到12.4 亿部，同比增长 1.6%。由于今年一二季度全球智能手机 出货量接近2 位数的增长，2025 年上半年智能手机出货量可能会录得同比 下滑。但是，明年同比增速有望呈现前低后高的态势，从而带来全年出货量 的正增长(图表30)。' metadata={'source': '科技行业 2025 年展望.txt', '作者': '沈岱,马智焱,黄佳琦', '发表时间': '2024 年 12 月 13 日', '文章标题': '科技行业2025年展望'}
********************************************************************************
page_content='其中，小米的高端机型表现比较典型。今年三季度，小米 3,000 元人民币以 上的智能手机出货量占到小米总出货量的6.3%，同比增加 1.5 个百分点，环 比增加0.5 个百分点(图表 41)。
展望四季度，国内和海外节假日、购物节集中，

'2024年三季度全球智能手机出货量为3.15亿部。'